In [15]:
import pandas as pd


dfNF = pd.read_csv('daily_average.csv')
dfWF = pd.read_csv('daily_averageNoFire.csv')

dfNFSD = pd.read_csv('NaturalFiresTOPOGRAPHY.csv')
dfWFSD = pd.read_csv('LocationsWFire.csv')

In [ ]:
print(dfNF.columns)
print(dfWF.columns)

temp = 28_100
hum = 7_28

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

columns_to_drop = ['day', 'index', 'Unnamed: 0', 'index.1']

# Drop the columns
df = dfNF.drop(columns=columns_to_drop)

correlation_matrix = df.corr()

plt.figure(figsize=(30, 26))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm')


plt.show()




In [ ]:
columns_to_keep = ['location', 'hourly.temperature_2m', 'hourly.relative_humidity_2m', 'hourly.precipitation', 'hourly.wind_speed_10m', 'hourly.soil_temperature_28_to_100cm', 'hourly.soil_moisture_7_to_28cm', 'hourly.direct_normal_irradiance']
df = dfNF[columns_to_keep]
dfW = dfWF[columns_to_keep]
print(df.columns)
print(dfW.columns)

In [ ]:
columns_to_keep = ['location', 'hourly.temperature_2m', 'hourly.relative_humidity_2m', 'hourly.precipitation', 'hourly.wind_speed_10m', 'hourly.soil_temperature_28_to_100cm', 'hourly.soil_moisture_7_to_28cm', 'hourly.direct_normal_irradiance']
df = dfNF[columns_to_keep]
dfW = dfWF[columns_to_keep]

df['wildfire'] = 1

df.set_index('location', inplace=True)

columns_days = {
    'hourly.temperature_2m': 8,
    'hourly.relative_humidity_2m': 8,
    'hourly.precipitation': 16,
    'hourly.wind_speed_10m': 16,
    'hourly.soil_temperature_28_to_100cm': 8,
    'hourly.soil_moisture_7_to_28cm': 8,
    'hourly.direct_normal_irradiance': 8
}

for column, days in columns_days.items():
    df = df.sort_index(ascending=False)
    df[f'{column}_lag'] = df.groupby(level=0)[column].shift(days)
    df = df.sort_index()

df.drop(columns=columns_days.keys(), inplace=True)

df.dropna(inplace=True)

# Without fire

dfW['wildfire'] = 0
dfW.set_index('location', inplace=True)

for column, days in columns_days.items():
    dfW = dfW.sort_index(ascending=False)
    dfW[f'{column}_lag'] = dfW.groupby(level=0)[column].shift(days)
    dfW = dfW.sort_index()

dfW.drop(columns=columns_days.keys(), inplace=True)
dfW.dropna(inplace=True)

print(df.columns)
print(dfW.columns)

dfNFSD['location'] = range(1, len(dfNFSD) + 1)
dfWFSD['location'] = range(1, len(dfWFSD) + 1)

# Convert 'CLC_CODE' to integer in dfNFSD and dfWFSD
dfNFSD['CLC_CODE'] = dfNFSD['CLC_CODE'].astype(int)
dfWFSD['CLC_CODE'] = dfWFSD['CLC_CODE'].astype(int)

print(dfNFSD.head())
print(dfWFSD.head())

#print(df.head())
#print(dfW.head())scientificNames


variables_to_add = ['roughness', 'aspect', 'slope', 'CLC_CODE']

# For df
df = df.merge(dfNFSD[variables_to_add], left_index=True, right_on='location', how='left')

# For dfW
dfW = dfW.merge(dfWFSD[variables_to_add], left_index=True, right_on='location', how='left')

In [16]:
# Este é o correto a executar

columns_to_keep = ['location', 'hourly.temperature_2m', 'hourly.relative_humidity_2m', 'hourly.precipitation', 'hourly.wind_speed_10m', 'hourly.soil_temperature_28_to_100cm', 'hourly.soil_moisture_7_to_28cm', 'hourly.direct_normal_irradiance']
variables_to_add = ['roughness', 'aspect', 'slope', 'CLC_CODE','scientificNames']

dfNFSD['location'] = range(1, len(dfNFSD) + 1)
dfWFSD['location'] = range(1, len(dfWFSD) + 1)

# Convert 'CLC_CODE' to integer in dfNFSD and dfWFSD
dfNFSD['CLC_CODE'] = dfNFSD['CLC_CODE'].astype(int)
dfWFSD['CLC_CODE'] = dfWFSD['CLC_CODE'].astype(int)

df = dfNF[columns_to_keep]
dfW = dfWF[columns_to_keep]

df['wildfire'] = 1
dfW['wildfire'] = 0

df.set_index('location', inplace=True)
dfW.set_index('location', inplace=True)

columns_days = {
    'hourly.temperature_2m': 8,
    'hourly.relative_humidity_2m': 8,
    'hourly.precipitation': 16,
    'hourly.wind_speed_10m': 16,
    'hourly.soil_temperature_28_to_100cm': 8,
    'hourly.soil_moisture_7_to_28cm': 8,
    'hourly.direct_normal_irradiance': 8
}

location_features = {}

for location in df.index.unique():
    location_features[location] = {}

    for column, days in columns_days.items():
        df.sort_index(ascending=True, inplace=True)
        last_days = df.loc[location][-days:]
        location_features[location][f'{column}_lag'] = last_days[column].values

    location_features[location]['wildfire'] = df.loc[location]['wildfire'].values[0]

for location in dfNFSD['location']:
    for variable in variables_to_add:
        location_features[location][variable] = dfNFSD.loc[dfNFSD['location'] == location, variable].values[0]

for location in dfW.index.unique():
    location_features[int(location)+1770] = {}

    for column, days in columns_days.items():
        dfW.sort_index(ascending=True, inplace=True)
        last_days = dfW.loc[location][-days:]
        location_features[int(location)+1770][f'{column}_lag'] = last_days[column].values

    location_features[int(location)+1770]['wildfire'] = dfW.loc[location]['wildfire'].values[0]

for location in dfWFSD['location']:
    for variable in variables_to_add:
        location_features[int(location)+1770][variable] = dfWFSD.loc[dfNFSD['location'] == location, variable].values[0]

/tmp/ipykernel_4185/407642764.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['wildfire'] = 1
/tmp/ipykernel_4185/407642764.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfW['wildfire'] = 0


In [72]:
import json
import numpy as np

# Convert numpy arrays to lists and numpy int64 to int
for location in location_features:
    for feature in location_features[location]:
        if isinstance(location_features[location][feature], np.ndarray):
            location_features[location][feature] = location_features[location][feature].tolist()
        elif isinstance(location_features[location][feature], np.int64):
            location_features[location][feature] = int(location_features[location][feature])

# Save the dictionary into a JSON file
with open('location_features.json', 'w') as f:
    json.dump(location_features, f)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

dfx = pd.DataFrame(location_features).T

# Split the DataFrame into features (X) and target (y)
X = dfx.drop('wildfire', axis=1)
y = dfx['wildfire']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Random Forest model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = clf.predict(X_test)

# Evaluate the model
print('Accuracy:', accuracy_score(y_test, y_pred))

ValueError: setting an array element with a sequence.

In [18]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer


columns_to_keep = ['location', 'hourly.temperature_2m', 'hourly.relative_humidity_2m', 'hourly.precipitation', 'hourly.wind_speed_10m', 'hourly.soil_temperature_28_to_100cm', 'hourly.soil_moisture_7_to_28cm', 'hourly.direct_normal_irradiance']
variables_to_add = ['roughness', 'aspect', 'slope', 'CLC_CODE','scientificNames']

dfNFSD['location'] = range(1, len(dfNFSD) + 1)
dfWFSD['location'] = range(1, len(dfWFSD) + 1)

# Convert 'CLC_CODE' to integer in dfNFSD and dfWFSD
dfNFSD['CLC_CODE'] = dfNFSD['CLC_CODE'].astype(int)
dfWFSD['CLC_CODE'] = dfWFSD['CLC_CODE'].astype(int)

df = dfNF[columns_to_keep]
dfW = dfWF[columns_to_keep]

df['wildfire'] = 1
dfW['wildfire'] = 0

df.set_index('location', inplace=True)
dfW.set_index('location', inplace=True)

columns_days = {
    'hourly.temperature_2m': 8,
    'hourly.relative_humidity_2m': 8,
    'hourly.precipitation': 16,
    'hourly.wind_speed_10m': 16,
    'hourly.soil_temperature_28_to_100cm': 8,
    'hourly.soil_moisture_7_to_28cm': 8,
    'hourly.direct_normal_irradiance': 8
}

location_features = {}

for location in df.index.unique():
    location_features[location] = {}

    for column, days in columns_days.items():
        df.sort_index(ascending=True, inplace=True)
        last_days = df.loc[location][-days:]
        location_features[location][f'{column}_lag'] = last_days[column].values

    location_features[location]['wildfire'] = df.loc[location]['wildfire'].values[0]

for location in dfNFSD['location']:
    for variable in variables_to_add:
        location_features[location][variable] = dfNFSD.loc[dfNFSD['location'] == location, variable].values[0]

for location in dfW.index.unique():
    location_features[int(location)+1770] = {}

    for column, days in columns_days.items():
        dfW.sort_index(ascending=True, inplace=True)
        last_days = dfW.loc[location][-days:]
        location_features[int(location)+1770][f'{column}_lag'] = last_days[column].values

    location_features[int(location)+1770]['wildfire'] = dfW.loc[location]['wildfire'].values[0]

for location in dfWFSD['location']:
    for variable in variables_to_add:
        location_features[int(location)+1770][variable] = dfWFSD.loc[dfNFSD['location'] == location, variable].values[0]


# Assuming 'scientificNames' is in dfNFSD
dfNFSD['scientificNames'] = dfNFSD['scientificNames'].str.split('; ')

# Use MultiLabelBinarizer to perform one-hot encoding
mlb = MultiLabelBinarizer()
tree_species_encoded = mlb.fit_transform(dfNFSD['scientificNames'])

# Convert the encoded tree species to a DataFrame
df_tree_species = pd.DataFrame(tree_species_encoded, columns=mlb.classes_, index=dfNFSD.index)

# Concatenate the original DataFrame with the tree species DataFrame
dfNFSD = pd.concat([dfNFSD, df_tree_species], axis=1)

# Now you can drop the original 'scientificNames' column
dfNFSD = dfNFSD.drop('scientificNames', axis=1)


dfWFSD['scientificNames'] = dfWFSD['scientificNames'].str.split('; ')

mlb = MultiLabelBinarizer()
tree_species_encoded = mlb.fit_transform(dfWFSD['scientificNames'])

df_tree_species = pd.DataFrame(tree_species_encoded, columns=mlb.classes_, index=dfNFSD.index)

dfWFSD = pd.concat([dfWFSD, df_tree_species], axis=1)

dfWFSD = dfWFSD.drop('scientificNames', axis=1)


# Initialize an empty DataFrame
dfx = pd.DataFrame()

# Loop over each location
for location, features in location_features.items():
    # Flatten the arrays and create new features
    for i, temp in enumerate(features['hourly.temperature_2m_lag']):
        dfx.loc[location, f'temperature_{i}'] = temp

    for i, humidity in enumerate(features['hourly.relative_humidity_2m_lag']):
        dfx.loc[location, f'humidity_{i}'] = humidity

    for i, precipitation in enumerate(features['hourly.precipitation_lag']):
        dfx.loc[location, f'precipitation_{i}'] = precipitation

    for i, wind_speed in enumerate(features['hourly.wind_speed_10m_lag']):
        dfx.loc[location, f'wind_speed_{i}'] = wind_speed

    for i, soil_temperature in enumerate(features['hourly.soil_temperature_28_to_100cm_lag']):
        dfx.loc[location, f'soil_temperature{i}'] = soil_temperature

    for i, soil_moisture in enumerate(features['hourly.soil_moisture_7_to_28cm_lag']):
        dfx.loc[location, f'soil_temperature{i}'] = soil_moisture

    for i, irradiance in enumerate(features['hourly.direct_normal_irradiance_lag']):
        dfx.loc[location, f'direct_normal_irradiance{i}'] = irradiance

    for variable in variables_to_add:
        dfx.loc[location, variable] = features[variable]

    dfx.loc[location, 'wildfire'] = features['wildfire']

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X = dfx.drop('wildfire', axis=1)
y = dfx['wildfire']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)

clf.fit(X_train, y_train)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

/tmp/ipykernel_4185/3004214000.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['wildfire'] = 1
/tmp/ipykernel_4185/3004214000.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfW['wildfire'] = 0


RandomForestClassifier(random_state=42)

In [21]:
print(X.columns)

Index(['temperature_0', 'temperature_1', 'temperature_2', 'temperature_3',
       'temperature_4', 'temperature_5', 'temperature_6', 'temperature_7',
       'humidity_0', 'humidity_1', 'humidity_2', 'humidity_3', 'humidity_4',
       'humidity_5', 'humidity_6', 'humidity_7', 'precipitation_0',
       'precipitation_1', 'precipitation_2', 'precipitation_3',
       'precipitation_4', 'precipitation_5', 'precipitation_6',
       'precipitation_7', 'precipitation_8', 'precipitation_9',
       'precipitation_10', 'precipitation_11', 'precipitation_12',
       'precipitation_13', 'precipitation_14', 'precipitation_15',
       'wind_speed_0', 'wind_speed_1', 'wind_speed_2', 'wind_speed_3',
       'wind_speed_4', 'wind_speed_5', 'wind_speed_6', 'wind_speed_7',
       'wind_speed_8', 'wind_speed_9', 'wind_speed_10', 'wind_speed_11',
       'wind_speed_12', 'wind_speed_13', 'wind_speed_14', 'wind_speed_15',
       'soil_temperature0', 'soil_temperature1', 'soil_temperature2',
       'soil_temper

In [11]:
# Make probability predictions on the testing set
y_pred_proba = clf.predict_proba(X_test)

# The result is a 2D array where the first element of each entry is the probability of class 0 and the second is the probability of class 1
# We are interested in the probability of class 1 (wildfire), so we select the second column with [:, 1]
y_pred_proba = y_pred_proba[:, 1]

# Now y_pred_proba contains the predicted probabilities of a wildfire
print(y_pred_proba)

[0.83 0.18 0.24 0.14 0.19 0.18 0.05 0.34 0.89 0.89 0.08 0.52 0.26 0.21
 0.36 0.12 0.84 0.91 0.04 0.07 0.49 0.21 0.79 0.18 0.18 0.2  0.56 0.4
 0.18 0.77 0.91 0.65 0.32 0.36 0.05 0.68 0.56 0.25 0.33 0.29 0.29 0.3
 0.68 0.95 0.74 0.44 0.9  0.41 0.18 0.89 0.31 0.14 0.11 0.4  0.39 0.69
 0.72 0.03 0.89 0.08 0.18 0.1  0.9  0.4  0.09 0.34 0.61 0.56 0.47 0.64
 0.8  0.99 0.14 0.59 0.69 0.73 0.07 0.48 0.48 0.77 0.9  0.44 0.59 0.66
 0.54 0.43 0.54 0.98 0.9  0.59 0.9  0.12 0.35 0.44 0.54 0.47 0.51 0.75
 0.32 0.26 0.9  0.56 0.13 0.6  0.44 0.31 0.28 0.34 0.13 0.64 0.95 0.33
 0.61 0.9  0.35 0.18 0.94 0.94 0.89 0.48 0.38 0.5  0.57 0.69 0.54 0.62
 0.98 0.92 0.27 0.69 0.56 0.47 0.57 0.73 0.75 0.59 0.16 0.65 0.11 0.25
 0.55 0.18 0.46 0.33 0.44 0.85 0.33 0.2  0.56 0.94 0.1  0.79 0.89 0.76
 1.   0.28 0.41 0.09 0.07 0.78 0.03 0.85 0.12 0.56 0.16 0.58 0.12 0.97
 0.64 0.52 0.25 0.19 0.26 0.2  0.64 0.41 0.07 0.58 0.77 0.71 0.27 0.
 0.87 0.75 0.36 0.43 0.82 0.98 0.08 0.11 0.87 0.71 0.11 0.88 0.48 0.67
 0.81 0.42

In [20]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

Precision: 0.8532608695652174
Recall: 0.8177083333333334
F1 Score: 0.8351063829787234


In [8]:
# Count the number of wildfires in the test set
num_wildfires = y_test.sum()

print('Number of wildfires in the test set:', num_wildfires)

Number of wildfires in the test set: 384.0


In [12]:
pd.set_option('display.max_rows', None)


y_pred_proba = clf.predict_proba(X_test)

# Select the probabilities of class 1 (wildfire)
y_pred_proba = y_pred_proba[:, 1]

df_results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred, 'Predicted_Probability': y_pred_proba})

df_results['Correct'] = df_results['Actual'] == df_results['Predicted']

print(df_results)

      Actual  Predicted  Predicted_Probability  Correct
713      1.0        1.0                   0.83     True
280      1.0        0.0                   0.18    False
1457     1.0        0.0                   0.24    False
3505     0.0        0.0                   0.14     True
2195     0.0        0.0                   0.19     True
2081     0.0        0.0                   0.18     True
3209     0.0        0.0                   0.05     True
1789     0.0        0.0                   0.34     True
479      1.0        1.0                   0.89     True
1072     1.0        1.0                   0.89     True
2288     0.0        0.0                   0.08     True
2177     0.0        1.0                   0.52    False
2910     0.0        0.0                   0.26     True
2571     0.0        0.0                   0.21     True
3361     0.0        0.0                   0.36     True
2414     0.0        0.0                   0.12     True
1736     1.0        1.0                   0.84  